In [1]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



In [2]:
import pandas as pd
import folium

In [17]:
import requests

request_parameters = {
    "client_id": 'LXAXANK2ZYUXHL2BSKZU3S0ABFHCUHS1LBLHH5SJDO3XZWGA',
    "client_secret": 'FJ3M4XBQFZOULXBRICY0ZZX02EZZFJFADCN1JCI2HZH12ZPC',
    "v": '20180323',
    "section": "coffee",
    "near": "Ahmedabad",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

Parsing response to JSON. We get to following keys in our dictionary:

In [18]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

Most of the items on dictionary are giving some general information like the name of the city, it's geo code and so on. `groups` contains the actual search result.

In [19]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Ahmedabad', 'Ahmedabad')

In [22]:
d["suggestedBounds"] 
d["totalResults"]

18

In [23]:
d["geocode"]

{'what': '',
 'where': 'ahmedabad',
 'center': {'lat': 23.02579, 'lng': 72.58727},
 'displayString': 'Ahmedabad, Gujarat, India',
 'cc': 'IN',
 'geometry': {'bounds': {'ne': {'lat': 23.167063007051212,
    'lng': 72.71177811117003},
   'sw': {'lat': 22.934192744971654, 'lng': 72.4286040158387}}},
 'slug': 'ahmadābād',
 'longId': '72057594039207169'}

In groups we again have several different items, where type and name are irrelevant for given task. The full path to items is `data.json()["response"]["groups"][0]["items"]`.

Let's print couple of first items to get again better idea of the structure of the data:

In [24]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [25]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [26]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 18


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b542b46f964a520b3b327e3',
  'name': 'Lucky Tea',
  'contact': {},
  'location': {'address': 'Lal Darwaja',
   'lat': 23.02782947436309,
   'lng': 72.5813939352504,
   'labeledLatLngs': [{'label': 'display',
     'lat': 23.02782947436309,
     'lng': 72.5813939352504}],
   'cc': 'IN',
   'city': 'Ahmedabad',
   'state': 'Gujarāt',
   'country': 'India',
   'formattedAddress': ['Lal Darwaja', 'Ahmedabad', 'Gujarāt', 'India']},
  'categories': [{'id': '4bf58dd8d48988d1dc931735',
    'name': 'Tea Room',
    'pluralName': 'Tea Rooms',
    'shortName': 'Tea Room',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tearoom_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 0,
   'la

In [27]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4bb964b053649c745b0448fb',
  'name': 'House of MG',
  'contact': {},
  'location': {'address': 'Lal Darwaja',
   'crossStreet': 'opposite Sidi Saiyad',
   'lat': 23.02718604857762,
   'lng': 72.58162130548759,
   'labeledLatLngs': [{'label': 'display',
     'lat': 23.02718604857762,
     'lng': 72.58162130548759}],
   'postalCode': '380001',
   'cc': 'IN',
   'city': 'Ahmedabad',
   'state': 'Gujarāt',
   'country': 'India',
   'formattedAddress': ['Lal Darwaja (opposite Sidi Saiyad)',
    'Ahmedabad 380001',
    'Gujarāt',
    'India']},
  'categories': [{'id': '4bf58dd8d48988d1fa931735',
    'name': 'Hotel',
    'pluralName': 'Hotels',
    'shortName': 'Hotel',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     'suffix': '.png'},
    'primary': True}],
  'verified': True,
  'stats': {'tipCoun

Based on this, a simple for loop over the items is collecting some relevant data from the array, and after that DataFrame is created based on this parsed data.

In [32]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
   # address = location["address"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

found 4 cafes


,uid,name,shortname,address,postalcode,lat,lng
0,4bb964b053649c745b0448fb,House of MG,Hotel,Lal Darwaja,380001,23.027186,72.581621
1,4ce92a7bd99f721e35c1b473,Cafe Natrani,Café,Lal Darwaja,380013,23.047775,72.572577
2,59c8ec0f018cbb55bff55634,Chai Gate,Tea Room,Lal Darwaja,380007,23.012164,72.569281
3,4f5ca1ebe4b0090c0a9f4a2a,Cafe Court @ SCG,Café,Lal Darwaja,380014,23.040948,72.564642


Some density based estimator is giving a good tip where to start a new coffee business. There's a `HeatMap` plugin ready in Folium, let's use that, and visualize all the existing Cafes to same map:

In [33]:
ahmedabad_center = d["geocode"]["center"]
ahmedabad_center

{'lat': 23.02579, 'lng': 72.58727}

In [36]:
from folium import plugins

# create map of Helsinki using latitude and longitude values
map_ahmedabad = folium.Map(location=[ahmedabad_center["lat"], ahmedabad_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_ahmedabad)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_ahmedabad.add_child(plugins.HeatMap(hm_data))

map_ahmedabad

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


## Results

Based on these results, one possibly good location for new Cafe would be near Gandhi Road

In [37]:
lat = 23.02579
lng = 72.58727
map_ahmedabad = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="New Cafe!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_ahmedabad)
map_ahmedabad